# Selects concrete beams with $\rho$ inside a range 

$\rho$ is determined from gross area ($A_g$), not $b_wd$

In [85]:
import pandas as pd
import numpy as np
import os
import sys
import comtypes.client
import lib.csiapi

import json
import tkinter as tk
from tkinter import filedialog
import lib.csiapi as csiapi

%load_ext autoreload
%autoreload 2

ETABSObject, SapModel = csiapi.get_active_etabs_object()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Succesfully created helper object
Succesfully created SapModel


In [87]:
SapModel.Analyze.RunAnalysis()
SapModel.DesignConcrete.StartDesign()

0

## Set bounds

Set lower bound and upper bound of $\rho$ 

In [89]:
as_percent_lb = 0
as_percent_ub = 0.5

In [99]:
table_key = 'Frame Section Property Definitions - Summary'
table = csiapi.get_table_display(SapModel,table_key)
table_simple = table[['Name','Material','Area']]
table_key = 'Concrete Beam Design Summary - ' + SapModel.DesignConcrete.GetCode()[0]
table2 = csiapi.get_table_display(SapModel,table_key)
table2_simple = table2[['Story','Label','UniqueName','DesignSect','AsTopTotal','AsBotTotal','AtTrnTotal']]

maximum_as = table2_simple.merge(table_simple, left_on = 'DesignSect', right_on = 'Name')
maximum_as = maximum_as.groupby(['UniqueName']).max()

maximum_as['AsTopPercentage'] = maximum_as['AsTopTotal'].div(maximum_as['Area'])*100
maximum_as['AsBotPercentage'] = maximum_as['AsBotTotal'].div(maximum_as['Area'])*100

beam_in_range = maximum_as[((maximum_as['AsTopPercentage'] > as_percent_lb) & 
                                       (maximum_as['AsTopPercentage'] < as_percent_ub)) | ((maximum_as['AsTopPercentage'] < as_percent_ub) & (maximum_as['AsBotPercentage'] > as_percent_lb))].reset_index()

uniquenames = pd.Series(beam_in_range['UniqueName']).unique()

# uniquenamelist = uniquenames.to_numpy()
for i in uniquenames:
    ret = SapModel.FrameObj.SetSelected(str(i.item()),True)

In [101]:
beam_in_range

,UniqueName,Story,Label,DesignSect,AsTopTotal,AsBotTotal,AtTrnTotal,Name,Material,Area,AsTopPercentage,AsBotPercentage
0,72,Lantai Atap,B305,G3050-3,0.000743,0.000433,0.00077,G3050-3,Concrete,0.15,0.495333,0.288667
1,73,Lantai Atap,B499,G3050-5,0.000700,0.000574,0.00143,G3050-5,Concrete,0.15,0.466667,0.382667
2,74,Lantai Atap,B107,G3050-3,0.000672,0.000490,0.00131,G3050-3,Concrete,0.15,0.448000,0.326667
3,75,Lantai Atap,B541,G3050-3,0.000645,0.000505,0.00141,G3050-3,Concrete,0.15,0.430000,0.336667
4,82,Lantai Atap,B150,G3050-5,0.000683,0.000558,0.00142,G3050-5,Concrete,0.15,0.455333,0.372000
...,...,...,...,...,...,...,...,...,...,...,...,...
205,1059,Lantai Atap,B270,B3050-3,0.000277,0.000032,0.00000,B3050-3,Concrete,0.15,0.184667,0.021333
206,1060,Lantai Atap,B278,B3050-3,0.000529,0.000125,0.00025,B3050-3,Concrete,0.15,0.352667,0.083333
207,1061,Lantai Atap,B269,G3050-3,0.000629,0.000459,0.00089,G3050-3,Concrete,0.15,0.419333,0.306000
208,1082,Lantai Atap,B180,B3060-6,0.000573,0.001428,0.00065,B3060-6,Concrete,0.18,0.318333,0.793333
